In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

Topic_Name1 = {topic1}
Topic_Name2 = {topic2}
Topic_Name3 = {topic3}
EH_NS_NAME = {Event Hb Name}
BOOTSTRAP_SERVERS = f"{EH_NS_NAME}.servicebus.windows.net:9093"
SAKEY = "UR+tdi5brOqFxphEl2rZdwszylRHA3tkwhOqsdqA464="
CONN_STRING = f"Endpoint=sb://{EH_NS_NAME}.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey={SAKEY}"
LOGIN_MODULE = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule"
EH_SASL = (
    f'{LOGIN_MODULE} required username="$ConnectionString" password="{CONN_STRING}";'
)

# --Create first bronze table--#

df1 = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", f"{EH_NS_NAME}.servicebus.windows.net:9093")
    .option("subscribe", Topic_Name1)  
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", EH_SASL)
    .option("kafka.request.timeout.ms", "60000")
    .option("kafka.session.timeout.ms", "60000")
    .option("failOnDataLoss", "false")
    .option("startingOffsets", "earliest")
    .load()
)

# Convert from base64 to string
df1 = df.withColumn(
    "key", col("key").cast("string")
).withColumn("value", col("value").cast("string"))

# Create raw delta live table
@dlt.table(
    table_properties={"pipelines.reset.allowed": "false"},
    spark_conf={"pipelines.trigger.interval": "1 seconds"},
)
def data1_raw():
    return df1

# --Create second bronze table--#
df2 = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", f"{EH_NS_NAME}.servicebus.windows.net:9093")
    .option("subscribe", Topic_Name2)  
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", EH_SASL)
    .option("kafka.request.timeout.ms", "60000")
    .option("kafka.session.timeout.ms", "60000")
    .option("failOnDataLoss", "false")
    .option("startingOffsets", "earliest")
    .load()
)

# Convert from base64 to string
df2 = df2.withColumn("key", col("key").cast("string")).withColumn(
    "value", col("value").cast("string")
)

# Create raw delta live table
@dlt.table(
    #     table_properties={"pipelines.reset.allowed": "false"},
    spark_conf={"pipelines.trigger.interval": "1 seconds"},
)
def data2_raw():
    return df2

# --Create third bronze table--#
df3 = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", f"{EH_NS_NAME}.servicebus.windows.net:9093")
    .option("subscribe", Topic_Name3)  
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", EH_SASL)
    .option("kafka.request.timeout.ms", "60000")
    .option("kafka.session.timeout.ms", "60000")
    .option("failOnDataLoss", "false")
    .option("startingOffsets", "earliest")
    .load()
)

# Convert from base64 to string
df3 = df3.withColumn("key", col("key").cast("string")).withColumn(
    "value", col("value").cast("string")
)

# Create raw delta live table
@dlt.table(
    table_properties={"pipelines.reset.allowed": "false"},
    spark_conf={"pipelines.trigger.interval": "1 seconds"},
)
def data3_raw():
    return df3